In [104]:
import matplotlib
%matplotlib inline
import numpy as np
from nilearn import datasets
from nilearn import plotting
import nibabel as nib

from neurolang import frontend as fe

Example of how to translate the sulci file

`blender_obj_to_gifti -i 101309_L.obj -r 101309.L.pial.32k_fs_LR.surf.gii -o 101309.L.manual_drawings.func.gii`

`blender_obj_to_gifti -i 101309_L.obj -r 101309.L.pial.32k_fs_LR.surf.gii -o 101309.L.manual_drawings.func.gii -p ".*(sulcus|fissure|Fissure|primus)*."`

In [105]:
destrieux_dataset = datasets.fetch_atlas_destrieux_2009()
destrieux_map = nib.load(destrieux_dataset['maps'])
destrieux_dataset['labels']

/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/numpy/lib/npyio.py:2278: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


rec.array([(  0, b'Background'), (  1, b'L G_and_S_frontomargin'),
           (  2, b'L G_and_S_occipital_inf'),
           (  3, b'L G_and_S_paracentral'),
           (  4, b'L G_and_S_subcentral'),
           (  5, b'L G_and_S_transv_frontopol'),
           (  6, b'L G_and_S_cingul-Ant'),
           (  7, b'L G_and_S_cingul-Mid-Ant'),
           (  8, b'L G_and_S_cingul-Mid-Post'),
           (  9, b'L G_cingul-Post-dorsal'),
           ( 10, b'L G_cingul-Post-ventral'), ( 11, b'L G_cuneus'),
           ( 12, b'L G_front_inf-Opercular'),
           ( 13, b'L G_front_inf-Orbital'),
           ( 14, b'L G_front_inf-Triangul'), ( 15, b'L G_front_middle'),
           ( 16, b'L G_front_sup'), ( 17, b'L G_Ins_lg_and_S_cent_ins'),
           ( 18, b'L G_insular_short'), ( 19, b'L G_occipital_middle'),
           ( 20, b'L G_occipital_sup'), ( 21, b'L G_oc-temp_lat-fusifor'),
           ( 22, b'L G_oc-temp_med-Lingual'),
           ( 23, b'L G_oc-temp_med-Parahip'), ( 24, b'L G_orbital'),
  

In [106]:
surface = nib.load('101309.R.pial.32k_fs_LR.surf.gii')
vertices = surface.darrays[0].data
manual_sulci_gii = nib.load('101309_R_manual_segmentation.func.gii')
manual_sulci = dict()
for darray in manual_sulci_gii.darrays:
    name = darray.meta.metadata['Name']
    name = name[:name.rfind('_')]
    points = vertices[darray.data.nonzero()]
    manual_sulci[name] = points
    print(f"{name}: number of points {points.shape[0]}")

Temporopolar_sulcus: number of points 38
Anterior_horizontal_ramus_lateral_fissure: number of points 57
Anterior_vertical_ramus_lateral_fissure: number of points 37
Angular_sulcus: number of points 112
Intermediate_primus_of_Jensen: number of points 209
Callosomarginal_sulcus: number of points 121
Orbital_H_shaped_sulcus: number of points 156
Olfactory_sulcus: number of points 96
Superior_rostral_sulcus: number of points 40
Superior_parietal_sulcus: number of points 145
Superior_occipital_sulcus: number of points 176
Superior_frontal_sulcus: number of points 420
Superior_temporal_sulcus: number of points 451
Subparietal_sulcus: number of points 199
Rhinal_sulcus: number of points 55
Precentral_sulcus: number of points 580
Postcentral_sulcus: number of points 634
Parieto_occipital_sulcus: number of points 208
Paracingulate_sulcus: number of points 683
Paracentral_sulcus: number of points 81
Occipitotemporal_sulcus: number of points 443
Anterior_occipital_sulcus: number of points 50
Midd

/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/parse_gifti_fast.py:62: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  newarr = np.fromstring(zdec, dtype=dt)
/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/gifti.py:760: ResourceWarning: unclosed file <_io.BufferedReader name='101309.R.pial.32k_fs_LR.surf.gii'>
  parser.parse(fptr=file_map['image'].get_prepare_fileobj('rb'))
/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/gifti.py:760: ResourceWarning: unclosed file <_io.BufferedReader name='101309_R_manual_segmentation.func.gii'>
  parser.parse(fptr=file_map['image'].get_prepare_fileobj('rb'))


In [107]:
nl = fe.RegionFrontend()

for name, points in manual_sulci.items():
    ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
    region = fe.ExplicitVBR(
        ijk_points,
        destrieux_map.affine,
        img_dim=destrieux_map.shape
    )
    nl.add_region(region, result_symbol_name=f'manual_{name}')

# Surfaces


In [108]:
Medial_surface_sulci = set()
Lateral_surface_sulci = set()
Ventral_surface_sulci = set()

for manual_sulcus_name in nl.region_names:
    if ('Callos' in manual_sulcus_name
        or 'Parieto_occipital' in manual_sulcus_name
        or 'Cingulate' in manual_sulcus_name 
        or 'alcarine' in manual_sulcus_name 
        or 'Marginal' in manual_sulcus_name
        or 'Paracingulate' in manual_sulcus_name
        or 'Superior_rostral' in manual_sulcus_name
        or 'Paracentral' in manual_sulcus_name
        or 'Subparietal' in manual_sulcus_name
        or 'Cuneal' in manual_sulcus_name
        or 'Intralingual' in manual_sulcus_name
        or 'Medial_frontal' in manual_sulcus_name):
        Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
    elif ('Central' in manual_sulcus_name
        or 'ateral_fissure' in manual_sulcus_name
        or 'Precentral' in manual_sulcus_name
        or 'Postcentral' in manual_sulcus_name 
        or 'Superior_frontal' in manual_sulcus_name 
        or 'Inferior_frontal' in manual_sulcus_name
        or 'Middle_frontal' in manual_sulcus_name
        or 'Frontomarginal' in manual_sulcus_name
        or 'subcentral' in manual_sulcus_name
        or 'Intraparietal' in manual_sulcus_name
        or 'Intermediate_primus' in manual_sulcus_name
        or 'Superior_parietal' in manual_sulcus_name
        or 'Superior_occipital' in manual_sulcus_name
        or 'Inferior_occipital' in manual_sulcus_name 
        or 'Lateral_occipital' in manual_sulcus_name 
        or 'Lunate' in manual_sulcus_name
        or 'Superior_temporal' in manual_sulcus_name
        or 'Inferior_temporal' in manual_sulcus_name
        or 'Anterior_occipital' in manual_sulcus_name
        or 'Angular' in manual_sulcus_name
        or 'Medial_frontal' in manual_sulcus_name):
        Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
    elif ('Occipitotemporal' in manual_sulcus_name
        or 'Collateral' in manual_sulcus_name
        or 'Rhinal' in manual_sulcus_name 
        or 'Parahippocampal' in manual_sulcus_name 
        or 'Hippocampal' in manual_sulcus_name
        or 'Olfactory' in manual_sulcus_name
        or 'Temporopolar' in manual_sulcus_name
        or 'Orbital' in manual_sulcus_name):
        Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
    else:
        print(manual_sulcus_name)

In [109]:
lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')

# Testing tolerance


### Anteriority/Posteriority

In [110]:
def tolerance_y(ys_origin_sulcus, ys_target_sulcus, length):
    # we consider I the origin
    before=set()
    overlaps=set()
    during=set()
    meets=set()
    starts=set()
    finishes=set()
    equals=set()
    J_minus = min(ys_target_sulcus)
    J_plus = max(ys_target_sulcus) + length
    
    I_boxes = set(ys_origin_sulcus)
    for x in I_boxes:
        I_minus = x
        I_plus = I_minus + length
        
        if I_minus < I_plus < J_minus < J_plus:
            before.add(x)
        if I_minus < J_minus < I_plus < J_plus:
            overlaps.add(x)
        if J_minus < I_minus < I_plus < J_plus:
            during.add(x)
        if I_minus < (I_plus==J_minus) < J_plus:
            meets.add(x)
        if (I_minus==J_minus) < I_plus < J_plus:
            starts.add(x)
        if J_minus < I_minus < (I_plus==J_plus):
            finishes.add(x)
        if (I_minus==J_minus) < (I_plus==J_plus):
            equals.add(x)
        
    
    print('before', len(before) / len(I_boxes) * 100)
    print('overlaps', len(overlaps) / len(I_boxes) * 100)
    print('during', len(during) / len(I_boxes) * 100)
    print('meets', len(meets) / len(I_boxes) * 100)
    print('starts', len(starts) / len(I_boxes) * 100)
    print('finishes', len(finishes) / len(I_boxes) * 100)
    print('equals', len(equals) / len(I_boxes) * 100)

In [111]:
def tolerance_y_2(ys_origin_sulcus, ys_target_sulcus, length):
    # we consider I the origin
    before=set()
    overlaps=set()
    during=set()
    meets=set()
    starts=set()
    finishes=set()
    equals=set()
    after=set()
    Anterior_of=set()
    Posterior_of=set()
    During=set()
    J_minus = min(ys_target_sulcus)
    print('J-:', J_minus)
    J_plus = max(ys_target_sulcus) + length
    print('J+:', J_plus)
    
    I_boxes = set(ys_origin_sulcus)
    for x in I_boxes:
        I_minus = x
        I_plus = I_minus + length
        
        if I_minus < I_plus < J_minus < J_plus:
            before.add(x)
        if I_minus < J_minus < I_plus < J_plus:
            overlaps.add(x)
        if J_minus < I_minus < I_plus < J_plus:
            during.add(x)
        if I_minus < I_plus == J_minus < J_plus:
            meets.add(x)
        if I_minus == J_minus < I_plus < J_plus:
            starts.add(x)
        if J_minus < I_minus < I_plus == J_plus:
            finishes.add(x)
        if I_minus == J_minus < I_plus == J_plus:
            equals.add(x)
        if J_minus < J_plus < I_minus < I_plus:
            after.add(x)
        
    
    print('before', len(before) / len(I_boxes) * 100)
    print('overlaps', len(overlaps) / len(I_boxes) * 100)
    print('during', len(during) / len(I_boxes) * 100)
    print('meets', len(meets) / len(I_boxes) * 100)
    print('starts', len(starts) / len(I_boxes) * 100)
    print('finishes', len(finishes) / len(I_boxes) * 100)
    print('equals', len(equals) / len(I_boxes) * 100)
    print('after', len(after) / len(I_boxes) * 100)
    
    if (len(before) / len(I_boxes))>=0.8:
        Anterior_of.add(str(ys_target_sulcus))
        return 'Anterior of'
    if (len(after) / len(I_boxes))>=0.8:
        Posterior_of.add(str(ys_target_sulcus))
        return 'Posterior of'
    if (len(during) / len(I_boxes))>=0.8:
        During.add(str(ys_target_sulcus))
        return 'During'
    else:
        return 'Threshold not reached'

In [112]:
cs_y = manual_sulci['Central_sulcus'].T[1]

In [113]:
call_y = manual_sulci['Callosal_sulcus'].T[1]

In [114]:
lf_y = manual_sulci['Lateral_fissure'].T[1]

### Lateral surface

In [115]:
lf_anteriority=tolerance_y_2(cs_y.round(1), lf_y.round(1), length=.1)

J-: -33.0
J+: 32.199998474121095
before 5.490196078431373
overlaps 0.39215686274509803
during 93.72549019607843
meets 0.0
starts 0.39215686274509803
finishes 0.0
equals 0.0
after 0.0


In [116]:
prcs_y = manual_sulci['Precentral_sulcus'].T[1]

In [117]:
prcs_anteriority_2= tolerance_y_2(cs_y.round(1), prcs_y.round(1), length=.1)
prcs_anteriority_2

J-: -21.3
J+: 13.6
before 43.13725490196079
overlaps 0.0
during 56.86274509803921
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 0.0


'Threshold not reached'

In [118]:
pocs_y = manual_sulci['Postcentral_sulcus'].T[1]

In [119]:
pocs_posteriority_2=tolerance_y_2(pocs_y.round(1), cs_y.round(1), length=.1)
pocs_posteriority_2

J-: -35.0
J+: 2.6999999046325684
before 44.106463878327
overlaps 0.38022813688212925
during 55.13307984790875
meets 0.0
starts 0.38022813688212925
finishes 0.0
equals 0.0
after 0.0


'Threshold not reached'

In [120]:
fms_y = manual_sulci['Frontomarginal_sulcus'].T[1]

In [121]:
fms_anteriority_2=tolerance_y_2(cs_y.round(1), fms_y.round(1), length=.1)
fms_anteriority_2

J-: 54.1
J+: 67.1
before 100.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 0.0


'Anterior of'

In [122]:
sfs_y = manual_sulci['Superior_frontal_sulcus'].T[1]

In [123]:
sfs_anteriority_2=tolerance_y_2(cs_y.round(1), sfs_y.round(1), length=.1)
sfs_anteriority_2

J-: -4.2
J+: 62.800000762939455
before 85.09803921568627
overlaps 0.0
during 14.50980392156863
meets 0.0
starts 0.39215686274509803
finishes 0.0
equals 0.0
after 0.0


'Anterior of'

In [124]:
sfs_from_prcs_anteriority_2=tolerance_y_2(prcs_y.round(1), sfs_y.round(1), length=.1)

J-: -4.2
J+: 62.800000762939455
before 43.873517786561266
overlaps 0.0
during 55.73122529644269
meets 0.0
starts 0.3952569169960474
finishes 0.0
equals 0.0
after 0.0


In [125]:
ifs_y = manual_sulci['Inferior_frontal_sulcus'].T[1]

In [126]:
ifs_anteriority_2=tolerance_y_2(cs_y.round(1), ifs_y.round(1), length=.1)
ifs_anteriority_2

J-: 6.6
J+: 63.199998474121095
before 100.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 0.0


'Anterior of'

In [127]:
ifs_from_prcs_anteriority_2=tolerance_y_2(prcs_y.round(1), ifs_y.round(1), length=.1)
ifs_from_prcs_anteriority_2

J-: 6.6
J+: 63.199998474121095
before 76.28458498023716
overlaps 0.3952569169960474
during 22.92490118577075
meets 0.0
starts 0.3952569169960474
finishes 0.0
equals 0.0
after 0.0


'Threshold not reached'

In [128]:
mfs_y = manual_sulci['Middle_frontal_sulcus'].T[1]

In [129]:
mfs_anteriority=tolerance_y_2(cs_y.round(1), mfs_y.round(1), length=.1)
mfs_anteriority

J-: 14.7
J+: 53.300000762939455
before 100.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 0.0


'Anterior of'

In [130]:
sps_y = manual_sulci['Superior_parietal_sulcus'].T[1]

In [131]:
sps_anteriority=tolerance_y_2(cs_y.round(1), sps_y.round(1), length=.1)
sps_anteriority

J-: -69.0
J+: -48.300001525878905
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [132]:
ipoj_y = manual_sulci['Intermediate_primus_of_Jensen'].T[1]

In [133]:
ipoj_anteriority_from_cs=tolerance_y_2(cs_y.round(1), ipoj_y.round(1), length=.1)
ipoj_anteriority_from_cs

J-: -52.3
J+: -39.49999847412109
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [134]:
ang_y=manual_sulci['Angular_sulcus'].T[1]

In [135]:
ang_anteriority_from_cs=tolerance_y_2(cs_y.round(1), ang_y.round(1), length=.1)
ang_anteriority_from_cs

J-: -71.0
J+: -51.800001525878905
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [136]:
sos_y=manual_sulci['Superior_occipital_sulcus'].T[1]

In [137]:
sos_anteriority_from_cs=tolerance_y_2(cs_y.round(1), sos_y.round(1), length=.1)
sos_anteriority_from_cs

J-: -90.1
J+: -59.60000076293945
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [138]:
ios_y=manual_sulci['Inferior_occipital_sulcus'].T[1]

In [139]:
ios_anteriority_from_cs=tolerance_y_2(cs_y.round(1), ios_y.round(1), length=.1)
ios_anteriority_from_cs

J-: -84.1
J+: -62.60000076293945
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [140]:
aos_y=manual_sulci['Anterior_occipital_sulcus'].T[1]

In [141]:
aos_anteriority_from_cs=tolerance_y_2(cs_y.round(1), aos_y.round(1), length=.1)
aos_anteriority_from_cs

J-: -69.5
J+: -52.9
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [142]:
its_y=manual_sulci['Inferior_temporal_sulcus'].T[1]

In [143]:
its_anteriority_from_cs=tolerance_y_2(cs_y.round(1), its_y.round(1), length=.1)
its_anteriority_from_cs

J-: -63.9
J+: 3.5000000953674317
before 0.0
overlaps 0.0
during 100.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 0.0


'During'

In [144]:
avrlf_y=manual_sulci['Anterior_vertical_ramus_lateral_fissure'].T[1]

In [145]:
avrlf_anteriority_from_cs=tolerance_y_2(cs_y.round(1), avrlf_y.round(1), length=.1)
avrlf_anteriority_from_cs

J-: 25.2
J+: 28.700000381469728
before 100.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 0.0


'Anterior of'

In [146]:
ahrlf_y=manual_sulci['Anterior_horizontal_ramus_lateral_fissure'].T[1]

In [147]:
ahrlf_anteriority_from_cs=tolerance_y_2(cs_y.round(1), ahrlf_y.round(1), length=.1)
ahrlf_anteriority_from_cs

J-: 30.4
J+: 37.199998474121095
before 100.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 0.0


'Anterior of'

In [148]:
avrlf_anteriority_2=tolerance_y_2(lf_y.round(1), avrlf_y.round(1), length=.1)
avrlf_anteriority_2

J-: 25.2
J+: 28.700000381469728
before 90.29411764705883
overlaps 0.0
during 2.941176470588235
meets 0.0
starts 0.0
finishes 0.29411764705882354
equals 0.0
after 6.470588235294119


'Anterior of'

In [149]:
ips_y=manual_sulci['Intraparietal_sulcus'].T[1]

In [150]:
ips_anteriority_2=tolerance_y_2(cs_y.round(1), ips_y.round(1), length=.1)
ips_anteriority_2

J-: -66.1
J+: -33.300001525878905
before 0.0
overlaps 0.0
during 4.313725490196078
meets 0.0
starts 0.0
finishes 0.39215686274509803
equals 0.0
after 95.29411764705881


'Posterior of'

In [151]:
ipoj_y=manual_sulci['Intermediate_primus_of_Jensen'].T[1]

In [152]:
ipoj_anteriority_2=tolerance_y_2(lf_y.round(1), ipoj_y.round(1), length=.1)
ipoj_anteriority_2

J-: -52.3
J+: -39.49999847412109
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [153]:
ang_anteriority_2=tolerance_y_2(lf_y.round(1), ang_y.round(1), length=.1)
ang_anteriority_2

J-: -71.0
J+: -51.800001525878905
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [154]:
sts_y=manual_sulci['Superior_temporal_sulcus'].T[1]

In [155]:
sts_anteriority_2=tolerance_y_2(lf_y.round(1), sts_y.round(1), length=.1)
sts_anteriority_2

J-: -52.8
J+: 21.300000762939455
before 0.0
overlaps 0.0
during 83.82352941176471
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 16.176470588235293


'During'

In [156]:
aos_of_its_posteriority_2=tolerance_y_2(its_y.round(1), aos_y.round(1), length=.1)
aos_of_its_posteriority_2

J-: -69.5
J+: -52.9
before 0.0
overlaps 0.0
during 7.804878048780488
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 92.19512195121952


'Posterior of'

In [157]:
aos_of_ios_anteriority_2=tolerance_y_2(ios_y.round(1), aos_y.round(1), length=.1)
aos_of_ios_anteriority_2

J-: -69.5
J+: -52.9
before 31.944444444444443
overlaps 1.3888888888888888
during 65.27777777777779
meets 0.0
starts 1.3888888888888888
finishes 0.0
equals 0.0
after 0.0


'Threshold not reached'

In [158]:
aos_of_ios_anteriority_2=tolerance_y_2(ios_y.round(1), aos_y.round(1), length=.1)
aos_of_ios_anteriority_2

J-: -69.5
J+: -52.9
before 31.944444444444443
overlaps 1.3888888888888888
during 65.27777777777779
meets 0.0
starts 1.3888888888888888
finishes 0.0
equals 0.0
after 0.0


'Threshold not reached'

In [159]:
aos_of_lf_anteriority_2=tolerance_y_2(lf_y.round(1), aos_y.round(1), length=.1)
aos_of_lf_anteriority_2

J-: -69.5
J+: -52.9
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [160]:
fms_of_lf_anteriority_2=tolerance_y_2(lf_y.round(1), fms_y.round(1), length=.1)
fms_of_lf_anteriority_2

J-: 54.1
J+: 67.1
before 100.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 0.0


'Anterior of'

In [161]:
ips_of_lf_anteriority_2=tolerance_y_2(lf_y.round(1), ips_y.round(1), length=.1)
ips_of_lf_anteriority_2

J-: -66.1
J+: -33.300001525878905
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [162]:
pocs_of_lf_anteriority_2=tolerance_y_2(lf_y.round(1), pocs_y.round(1), length=.1)
pocs_of_lf_anteriority_2

J-: -49.2
J+: -13.4
before 0.0
overlaps 0.0
during 37.94117647058823
meets 0.0
starts 0.0
finishes 0.29411764705882354
equals 0.0
after 61.76470588235294


'Threshold not reached'

In [163]:
sps_of_lf_anteriority_2=tolerance_y_2(lf_y.round(1), sps_y.round(1), length=.1)
sps_of_lf_anteriority_2

J-: -69.0
J+: -48.300001525878905
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [164]:
sos_of_lf_anteriority_2=tolerance_y_2(lf_y.round(1), sos_y.round(1), length=.1)
sos_of_lf_anteriority_2

J-: -90.1
J+: -59.60000076293945
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [165]:
ios_of_lf_anteriority_2=tolerance_y_2(lf_y.round(1), ios_y.round(1), length=.1)
ios_of_lf_anteriority_2

J-: -84.1
J+: -62.60000076293945
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [166]:
sps_of_lf_anteriority_2=tolerance_y_2(lf_y.round(1), sps_y.round(1), length=.1)
sps_of_lf_anteriority_2

J-: -69.0
J+: -48.300001525878905
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


'Posterior of'

In [167]:
sos_posterior=tolerance_y_2(ips_y.round(1), sos_y.round(1), length=0.1)
sos_posterior

J-: -90.1
J+: -59.60000076293945
before 0.0
overlaps 0.0
during 31.097560975609756
meets 0.0
starts 0.0
finishes 0.6097560975609756
equals 0.0
after 68.29268292682927


'Threshold not reached'

### Medial surface

In [168]:
pos_y = manual_sulci['Parieto_occipital_sulcus'].T[1]

In [169]:
pos_anteriority=tolerance_y_2(call_y.round(1), pos_y.round(1), length=.1)

J-: -78.5
J+: -57.4
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


In [170]:
call_anteriority_2=tolerance_y_2(cs_y.round(1), call_y.round(1), length=.1)

J-: -45.9
J+: 36.89999923706055
before 0.0
overlaps 0.0
during 100.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 0.0


In [171]:
calc_y = manual_sulci['Calcarine_sulcus'].T[1]

In [172]:
calc_anteriority_2=tolerance_y_2(call_y.round(1), calc_y.round(1), length=.1)

J-: -95.3
J+: -41.99999847412109
before 0.0
overlaps 0.0
during 9.688581314878892
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 89.96539792387543


In [173]:
cing_y = manual_sulci['Cingulate_sulcus'].T[1]

In [174]:
cing_anteriority_2=tolerance_y_2(cs_y.round(1), cing_y.round(1), length=.1)

J-: -36.1
J+: 45.6
before 0.0
overlaps 0.0
during 100.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 0.0


In [175]:
paracing_y = manual_sulci['Paracingulate_sulcus'].T[1]

In [176]:
paracing_anteriority_2=tolerance_y_2(cs_y.round(1), paracing_y.round(1), length=.1)

J-: -17.1
J+: 68.1
before 49.411764705882355
overlaps 0.0
during 50.19607843137255
meets 0.0
starts 0.39215686274509803
finishes 0.0
equals 0.0
after 0.0


In [177]:
srs_y = manual_sulci['Superior_rostral_sulcus'].T[1]

In [178]:
srs_anteriority_2=tolerance_y_2(cs_y.round(1), srs_y.round(1), length=.1)

J-: 40.6
J+: 60.199998474121095
before 100.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 0.0


In [179]:
subp_y = manual_sulci['Subparietal_sulcus'].T[1]

In [180]:
subp_posteriority=tolerance_y_2(call_y.round(1), subp_y.round(1), length=.1)

J-: -68.8
J+: -34.10000076293945
before 0.0
overlaps 0.0
during 19.72318339100346
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 80.27681660899654


In [181]:
cun_y = manual_sulci['Cuneal_sulcus'].T[1]

In [182]:
cun_anteriority=tolerance_y_2(call_y.round(1), cun_y.round(1), length=.1)

J-: -85.8
J+: -75.9
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


In [183]:
ils_y = manual_sulci['Intralingual_sulcus'].T[1]

In [184]:
ils_anteriority=tolerance_y_2(call_y.round(1), ils_y.round(1), length=.1)

J-: -79.8
J+: -63.49999847412109
before 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 100.0


In [185]:
marg_y=manual_sulci['Callosomarginal_sulcus'].T[1]

In [186]:
marg_posteriority=tolerance_y_2(cing_y.round(1), marg_y.round(1), length=.1)

J-: -43.5
J+: -34.699999237060545
before 0.0
overlaps 0.0
during 1.3192612137203166
meets 0.0
starts 0.0
finishes 0.2638522427440633
equals 0.0
after 98.41688654353563


### Superiority/Inferiority

In [187]:
def tolerance_z(zs_origin_sulcus, zs_target_sulcus, length):
    # we consider I the origin
    above=set()
    overlaps=set()
    during=set()
    meets=set()
    starts=set()
    finishes=set()
    equals=set()
    below=set()
    Superior_of=set()
    Inferior_of=set()
    During=set()
    J_minus = min(zs_target_sulcus)
    print('J-:', J_minus)
    J_plus = max(zs_target_sulcus) + length
    print('J+:', J_plus)
    
    I_boxes = set(zs_origin_sulcus)
    for x in I_boxes:
        I_minus = x
        I_plus = I_minus + length
        
        if I_minus < I_plus < J_minus < J_plus:
            above.add(x)
        if I_minus < J_minus < I_plus < J_plus:
            overlaps.add(x)
        if J_minus < I_minus < I_plus < J_plus:
            during.add(x)
        if I_minus < I_plus == J_minus < J_plus:
            meets.add(x)
        if I_minus == J_minus < I_plus < J_plus:
            starts.add(x)
        if J_minus < I_minus < I_plus == J_plus:
            finishes.add(x)
        if I_minus == J_minus < I_plus == J_plus:
            equals.add(x)
        if J_minus < J_plus < I_minus < I_plus:
            below.add(x)
        
    
    print('above', len(above) / len(I_boxes) * 100)
    print('overlaps', len(overlaps) / len(I_boxes) * 100)
    print('during', len(during) / len(I_boxes) * 100)
    print('meets', len(meets) / len(I_boxes) * 100)
    print('starts', len(starts) / len(I_boxes) * 100)
    print('finishes', len(finishes) / len(I_boxes) * 100)
    print('equals', len(equals) / len(I_boxes) * 100)
    print('below', len(below) / len(I_boxes) * 100)
    
    if (len(above) / len(I_boxes))>=0.8:
        Superior_of.add(str(zs_target_sulcus))
        return 'Superior of'
    if (len(below) / len(I_boxes))>=0.8:
        Inferior_of.add(str(zs_target_sulcus))
        return 'Inferior of'
    if (len(during) / len(I_boxes))>=0.8:
        During.add(str(zs_target_sulcus))
        return 'During'
    else:
        return 'Threshold not reached'

In [188]:
cs_z = manual_sulci['Central_sulcus'].T[2]

In [189]:
lf_z = manual_sulci['Lateral_fissure'].T[2]

In [190]:
sts_z = manual_sulci['Superior_temporal_sulcus'].T[2]

In [191]:
lf_inferior_from_cs=tolerance_z(cs_z.round(1), lf_z.round(1), length=0.1)
lf_inferior_from_cs

J-: 0.1
J+: 36.39999923706055
above 0.0
overlaps 0.0
during 40.3954802259887
meets 0.0
starts 0.0
finishes 0.2824858757062147
equals 0.0
below 59.32203389830508


'Threshold not reached'

In [192]:
sts_inferior=tolerance_z(lf_z.round(1), sts_z.round(1), length=0.1)
sts_inferior

J-: -40.2
J+: 24.300000762939455
above 0.0
overlaps 0.0
during 73.54085603112841
meets 0.0
starts 0.0
finishes 0.38910505836575876
equals 0.0
below 25.680933852140075


'Threshold not reached'

In [193]:
sts_inferior_from_cs=tolerance_z(cs_z.round(1), sts_z.round(1), length=0.1)
sts_inferior_from_cs

J-: -40.2
J+: 24.300000762939455
above 0.0
overlaps 0.0
during 16.38418079096045
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 83.61581920903954


'Inferior of'

In [194]:
its_z = manual_sulci['Inferior_temporal_sulcus'].T[2]

In [195]:
its_inferior=tolerance_z(lf_z.round(1), its_z.round(1), length=0.1)
its_inferior

J-: -37.2
J+: -5.9
above 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 100.0


'Inferior of'

In [196]:
its_inferior_from_cs=tolerance_z(cs_z.round(1), its_z.round(1), length=0.1)
its_inferior_from_cs

J-: -37.2
J+: -5.9
above 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 100.0


'Inferior of'

In [197]:
ips_z = manual_sulci['Intraparietal_sulcus'].T[2]

In [198]:
ips_superior=tolerance_z(lf_z.round(1), ips_z.round(1), length=0.1)
ips_superior

J-: 36.9
J+: 63.6
above 100.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 0.0


'Superior of'

In [199]:
ipoj_z = manual_sulci['Intermediate_primus_of_Jensen'].T[2]

In [200]:
ipoj_inferior=tolerance_z(ips_z.round(1), ipoj_z.round(1), length=0.1)
ipoj_inferior

J-: 21.8
J+: 47.199998474121095
above 0.0
overlaps 0.0
during 54.43037974683544
meets 0.0
starts 0.0
finishes 0.6329113924050633
equals 0.0
below 44.936708860759495


'Threshold not reached'

In [201]:
ipoj_inferior_from_cs=tolerance_z(cs_z.round(1), ipoj_z.round(1), length=0.1)
ipoj_inferior_from_cs

J-: 21.8
J+: 47.199998474121095
above 13.559322033898304
overlaps 0.2824858757062147
during 42.93785310734463
meets 0.0
starts 0.2824858757062147
finishes 0.2824858757062147
equals 0.0
below 42.65536723163842


'Threshold not reached'

In [202]:
ang_z = manual_sulci['Angular_sulcus'].T[2]

In [203]:
ang_inferior=tolerance_z(ips_z.round(1), ang_z.round(1), length=0.1)
ang_inferior

J-: 23.3
J+: 52.699998474121095
above 0.0
overlaps 0.0
during 83.54430379746836
meets 0.0
starts 0.0
finishes 0.6329113924050633
equals 0.0
below 15.822784810126583


'During'

In [204]:
ang_inferior_from_cs=tolerance_z(cs_z.round(1), ang_z.round(1), length=0.1)
ang_inferior_from_cs

J-: 23.3
J+: 52.699998474121095
above 15.254237288135593
overlaps 0.0
during 53.672316384180796
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 31.07344632768362


'Threshold not reached'

In [205]:
sos_z = manual_sulci['Superior_occipital_sulcus'].T[2]

In [206]:
sos_inferior=tolerance_z(ips_z.round(1), sos_z.round(1), length=0.1)
sos_inferior

J-: 9.4
J+: 43.699998474121095
above 0.0
overlaps 0.0
during 34.177215189873415
meets 0.0
starts 0.0
finishes 0.6329113924050633
equals 0.0
below 65.18987341772153


'Threshold not reached'

In [207]:
sos_inferior_from_cs=tolerance_z(cs_z.round(1), sos_z.round(1), length=0.1)
sos_inferior_from_cs

J-: 9.4
J+: 43.699998474121095
above 0.5649717514124294
overlaps 0.0
during 50.847457627118644
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 48.58757062146893


'Threshold not reached'

In [208]:
sos_inferior_from_lf=tolerance_z(lf_z.round(1), sos_z.round(1), length=0.1)
sos_inferior_from_lf

J-: 9.4
J+: 43.699998474121095
above 20.622568093385212
overlaps 0.38910505836575876
during 78.59922178988327
meets 0.0
starts 0.38910505836575876
finishes 0.0
equals 0.0
below 0.0


'Threshold not reached'

In [209]:
ios_z = manual_sulci['Inferior_occipital_sulcus'].T[2]

In [210]:
ios_inferior=tolerance_z(sos_z.round(1), ios_z.round(1), length=0.1)
ios_inferior

J-: -7.4
J+: 8.1
above 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 100.0


'Inferior of'

In [211]:
ios_inferior_from_cs=tolerance_z(cs_z.round(1), ios_z.round(1), length=0.1)
ios_inferior_from_cs

J-: -7.4
J+: 8.1
above 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 100.0


'Inferior of'

In [212]:
ios_inferior_from_lf=tolerance_z(lf_z.round(1), ios_z.round(1), length=0.1)
ios_inferior_from_lf

J-: -7.4
J+: 8.1
above 0.0
overlaps 0.0
during 16.342412451361866
meets 0.0
starts 0.0
finishes 0.38910505836575876
equals 0.0
below 83.26848249027238


'Inferior of'

In [213]:
los_z = manual_sulci['Lateral_occipital_sulcus'].T[2]

In [214]:
los_inferior=tolerance_z(sos_z.round(1), los_z.round(1), length=0.1)
los_inferior

J-: -1.3
J+: 4.400000190734863
above 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 100.0


'Inferior of'

In [215]:
los_inferior_from_lf=tolerance_z(lf_z.round(1), los_z.round(1), length=0.1)
los_inferior_from_lf

J-: -1.3
J+: 4.400000190734863
above 0.0
overlaps 0.0
during 5.836575875486381
meets 0.0
starts 0.0
finishes 0.38910505836575876
equals 0.0
below 93.77431906614785


'Inferior of'

In [216]:
los_inferior_from_cs=tolerance_z(cs_z.round(1), los_z.round(1), length=0.1)
los_inferior_from_cs

J-: -1.3
J+: 4.400000190734863
above 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 100.0


'Inferior of'

In [217]:
los_inferior_from_ios=tolerance_z(ios_z.round(1), los_z.round(1), length=0.1)
los_inferior_from_ios

J-: -1.3
J+: 4.400000190734863
above 40.0
overlaps 0.0
during 25.71428571428571
meets 0.0
starts 0.0
finishes 1.4285714285714286
equals 0.0
below 31.428571428571427


'Threshold not reached'

In [218]:
ifs_z = manual_sulci['Inferior_frontal_sulcus'].T[2]

In [219]:
ifs_superiority = tolerance_z(lf_z.round(1), ifs_z.round(1), length=0.1)
ifs_superiority

J-: -16.2
J+: 39.199998474121095
above 0.0
overlaps 0.0
during 100.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 0.0


'During'

In [220]:
ifs_inferiority_from_cs = tolerance_z(cs_z.round(1), ifs_z.round(1), length=0.1)
ifs_inferiority_from_cs

J-: -16.2
J+: 39.199998474121095
above 0.0
overlaps 0.0
during 45.76271186440678
meets 0.0
starts 0.0
finishes 0.2824858757062147
equals 0.0
below 53.954802259887


'Threshold not reached'

In [221]:
sfs_z = manual_sulci['Superior_frontal_sulcus'].T[2]

In [222]:
sfs_superiority = tolerance_z(lf_z.round(1), sfs_z.round(1), length=0.1)
sfs_superiority

J-: 10.0
J+: 53.1
above 22.957198443579767
overlaps 0.0
during 76.65369649805449
meets 0.0
starts 0.38910505836575876
finishes 0.0
equals 0.0
below 0.0


'Threshold not reached'

In [223]:
sfs_superiority_from_cs = tolerance_z(cs_z.round(1), sfs_z.round(1), length=0.1)
sfs_superiority_from_cs

J-: 10.0
J+: 53.1
above 0.5649717514124294
overlaps 0.0
during 68.92655367231639
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 30.508474576271187


'Threshold not reached'

In [224]:
ang_superiority_from_cs = tolerance_z(cs_z.round(1), ang_z.round(1), length=0.1)
ang_superiority_from_cs

J-: 23.3
J+: 52.699998474121095
above 15.254237288135593
overlaps 0.0
during 53.672316384180796
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 31.07344632768362


'Threshold not reached'

In [225]:
ang_superiority_from_lf = tolerance_z(lf_z.round(1), ang_z.round(1), length=0.1)
ang_superiority_from_lf

J-: 23.3
J+: 52.699998474121095
above 69.64980544747081
overlaps 0.38910505836575876
during 29.571984435797667
meets 0.0
starts 0.38910505836575876
finishes 0.0
equals 0.0
below 0.0


'Threshold not reached'

In [226]:
sfs_superiority_from_lf = tolerance_z(lf_z.round(1), sfs_z.round(1), length=0.1)
sfs_superiority_from_lf

J-: 10.0
J+: 53.1
above 22.957198443579767
overlaps 0.0
during 76.65369649805449
meets 0.0
starts 0.38910505836575876
finishes 0.0
equals 0.0
below 0.0


'Threshold not reached'

In [227]:
sfs_superiority_from_cs = tolerance_z(cs_z.round(1), sfs_z.round(1), length=0.1)
sfs_superiority_from_cs

J-: 10.0
J+: 53.1
above 0.5649717514124294
overlaps 0.0
during 68.92655367231639
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 30.508474576271187


'Threshold not reached'

In [228]:
ifs_superiority_from_lf = tolerance_z(lf_z.round(1), ifs_z.round(1), length=0.1)
ifs_superiority_from_lf

J-: -16.2
J+: 39.199998474121095
above 0.0
overlaps 0.0
during 100.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 0.0


'During'

In [229]:
ifs_superiority_from_cs = tolerance_z(cs_z.round(1), ifs_z.round(1), length=0.1)
ifs_superiority_from_cs

J-: -16.2
J+: 39.199998474121095
above 0.0
overlaps 0.0
during 45.76271186440678
meets 0.0
starts 0.0
finishes 0.2824858757062147
equals 0.0
below 53.954802259887


'Threshold not reached'

In [230]:
fms_z = manual_sulci['Frontomarginal_sulcus'].T[2]

In [231]:
fms_superiority_from_cs = tolerance_z(cs_z.round(1), fms_z.round(1), length=0.1)
fms_superiority_from_cs

J-: -10.8
J+: -0.700000011920929
above 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 100.0


'Inferior of'

In [232]:
fms_superiority_from_lf = tolerance_z(lf_z.round(1), fms_z.round(1), length=0.1)
fms_superiority_from_lf

J-: -10.8
J+: -0.700000011920929
above 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 100.0


'Inferior of'

In [233]:
avrlf_z = manual_sulci['Anterior_vertical_ramus_lateral_fissure'].T[2]
ahrlf_z = manual_sulci['Anterior_horizontal_ramus_lateral_fissure'].T[2]

In [234]:
avrlf_superiority_from_lf = tolerance_z(lf_z.round(1), avrlf_z.round(1), length=0.1)
avrlf_superiority_from_lf

J-: -1.9
J+: 1.4999999761581422
above 0.0
overlaps 0.0
during 2.3346303501945527
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 97.66536964980544


'Inferior of'

In [235]:
avrlf_superiority_from_cs = tolerance_z(cs_z.round(1), avrlf_z.round(1), length=0.1)
avrlf_superiority_from_cs

J-: -1.9
J+: 1.4999999761581422
above 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 100.0


'Inferior of'

In [236]:
ahrlf_superiority_from_lf = tolerance_z(lf_z.round(1), ahrlf_z.round(1), length=0.1)
ahrlf_superiority_from_lf

J-: -10.2
J+: 1.200000023841858
above 0.0
overlaps 0.0
during 1.556420233463035
meets 0.0
starts 0.0
finishes 0.38910505836575876
equals 0.0
below 98.0544747081712


'Inferior of'

In [237]:
ahrlf_superiority_from_cs = tolerance_z(cs_z.round(1), ahrlf_z.round(1), length=0.1)
ahrlf_superiority_from_cs

J-: -10.2
J+: 1.200000023841858
above 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 100.0


'Inferior of'

In [238]:
mfs_z = manual_sulci['Middle_frontal_sulcus'].T[2]

In [239]:
mfs_superiority_from_cs = tolerance_z(lf_z.round(1), mfs_z.round(1), length=0.1)
mfs_superiority_from_cs

J-: 15.6
J+: 58.6
above 42.4124513618677
overlaps 0.0
during 57.19844357976653
meets 0.0
starts 0.38910505836575876
finishes 0.0
equals 0.0
below 0.0


'Threshold not reached'

In [240]:
aos_z = manual_sulci['Anterior_occipital_sulcus'].T[2]

In [241]:
aos_inferiority_from_cs = tolerance_z(lf_z.round(1), aos_z.round(1), length=0.1)
aos_inferiority_from_cs

J-: -8.9
J+: -4.999999904632569
above 0.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 100.0


'Inferior of'

In [242]:
prcs_z = manual_sulci['Precentral_sulcus'].T[2]

In [243]:
cs_y_z = [cs_y, cs_z]
prcs_y_z=[prcs_y, cs_z]
lf_y_z=[lf_y, lf_z]
ifs_y_z=[ifs_y, ifs_z]

##### From CS and LF, can find IPS

In [244]:
if ips_anteriority_2 == 'Posterior of' and ips_superior == 'Superior of':
    print('IPS is posterior of CS and superior of LF')
else:
    print('try again')

IPS is posterior of CS and superior of LF


In [245]:
prcs_from_ips = tolerance_y_2(ips_y.round(1), prcs_y.round(1), length=0.1)
prcs_from_ips

J-: -21.3
J+: 13.6
before 100.0
overlaps 0.0
during 0.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
after 0.0


'Anterior of'

In [246]:
prcs_superiority = tolerance_z(lf_z.round(1), prcs_z.round(1), length=0.1)
prcs_superiority

J-: -0.2
J+: 74.1
above 0.0
overlaps 0.0
during 100.0
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 0.0


'During'

In [247]:
pocs_from_prcs = tolerance_y_2(prcs_y.round(1), pocs_y.round(1), length=0.1)
pocs_from_prcs

J-: -49.2
J+: -13.4
before 0.0
overlaps 0.0
during 12.648221343873518
meets 0.0
starts 0.0
finishes 0.3952569169960474
equals 0.0
after 86.95652173913044


'Posterior of'

In [248]:
ifs_inf_from_sfs = tolerance_z(sfs_z.round(1), ifs_z.round(1), length=0.1)
ifs_inf_from_sfs     

J-: -16.2
J+: 39.199998474121095
above 0.0
overlaps 0.0
during 61.41078838174274
meets 0.0
starts 0.0
finishes 0.0
equals 0.0
below 38.589211618257266


'Threshold not reached'